In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
# data_file = open('intents.json').read()
# intents = json.loads(data_file)
intents={"intents": [
        {"tag": "greeting",
         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],
         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],
         "context_set": ""
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye"],
         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful"],
         "responses": ["Happy to help!", "Any time!", "My pleasure"]
        },
        {"tag": "createwebsite",
         "patterns": ["I want to create a website.", "Start making a new website", "create a new site." ],
         "responses": ["Sure! Tell me more about it", "Ok. What kind of website do you want?"]
        },
        {"tag": "payments",
         "patterns": ["Do you take credit cards?", "Do you accept Mastercard?", "Are you cash only?" ],
         "responses": ["We accept VISA, Mastercard and AMEX", "We accept most major credit cards"]
        },
        {"tag": "opentoday",
         "patterns": ["Are you open today?", "When do you open today?", "What are your hours today?"],
         "responses": ["We're open every day from 9am-9pm", "Our hours are 9am-9pm every day"]
        }
   ]
}

In [ ]:

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

20 documents
6 classes ['createwebsite', 'goodbye', 'greeting', 'opentoday', 'payments', 'thanks']
44 unique lemmatized words ["'s", '.', 'a', 'accept', 'anyone', 'are', 'bye', 'card', 'cash', 'create', 'credit', 'day', 'do', 'good', 'goodbye', 'hello', 'helpful', 'hi', 'hour', 'how', 'i', 'is', 'later', 'making', 'mastercard', 'new', 'only', 'open', 'see', 'site', 'start', 'take', 'thank', 'thanks', 'that', 'there', 'to', 'today', 'want', 'website', 'what', 'when', 'you', 'your']


In [ ]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-5-1a891a7a8859>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
4/4 [==============================] - 1s 8ms/step - loss: 1.7274 - accuracy: 0.1500
Epoch 2/200
4/4 [==============================] - 0s 5ms/step - loss: 1.7421 - accuracy: 0.3000
Epoch 3/200
4/4 [==============================] - 0s 6ms/step - loss: 1.7294 - accuracy: 0.3000
Epoch 4/200
4/4 [==============================] - 0s 4ms/step - loss: 1.5933 - accuracy: 0.3500
Epoch 5/200
4/4 [==============================] - 0s 4ms/step - loss: 1.6926 - accuracy: 0.4000
Epoch 6/200
4/4 [==============================] - 0s 7ms/step - loss: 1.5412 - accuracy: 0.4500
Epoch 7/200
4/4 [==============================] - 0s 5ms/step - loss: 1.5462 - accuracy: 0.5500
Epoch 8/200
4/4 [==============================] - 0s 7ms/step - loss: 1.3952 - accuracy: 0.5500
Epoch 9/200
4/4 [==============================] - 0s 6ms/step - loss: 1.4428 - accuracy: 0.5000
Epoch 10/200
4/4 [==============================] - 0s 5ms/step - loss: 1.3306 - accuracy: 0.6500
Epoch 11/200
4/4 [===========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
while True:
  msg=input('You: ')
  if msg=='exit':
    break
  resp=chatbot_response(msg)
  print(f'Jarvis: {resp}')

You: hey
1/1 [==============================] - 0s 93ms/step
Jarvis: Hi there, how can I help?
You: I want to build a website
1/1 [==============================] - 0s 30ms/step
Jarvis: Sure! Tell me more about it
You: Do you use card?
1/1 [==============================] - 0s 24ms/step
Jarvis: We accept most major credit cards
You: goodbye
1/1 [==============================] - 0s 19ms/step
Jarvis: Have a nice day
You: exit
